## Cencepts from paper

"Measuring Semantic Similarity between Concepts in Visual Domain" Table 1

In [1]:
concepts = [
    "bay",
    "beach",
    "birds",
    "boeing",
    "buildings",
    "city",
    "clouds",
    "face",
    "f-16",
    "helicopter",
    "mountain",
    "sky",
    "ships",
    "sunset",
    "sunrise",
    "ocean"
]

In [2]:
len(concepts)

16

In [3]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options

In [4]:
chrome_options = Options()
chrome_options.binary_location = '/usr/bin/google-chrome'
chrome_options.add_argument("--headless")  


In [5]:
driver = webdriver.Chrome(chrome_options=chrome_options, executable_path="/usr/local/bin/chromedriver")

In [6]:
# selenium test.

# driver.get('http://www.google.com')
# print(driver.title)
# driver.close()

In [6]:
top_url = "http://images.search.yahoo.com/"

In [7]:
driver.get(top_url)

In [8]:
input = driver.find_element_by_id('yschsp')

In [9]:
input.send_keys('bay')

In [10]:
btn = driver.find_element_by_class_name('ygbt')

In [11]:
btn.click()

In [12]:
driver.title

'bay - Yahoo Image Search Results'

In [13]:
first_res = driver.find_element_by_id("resitem-0")

In [14]:
par = first_res.parent

In [15]:
all_lis = par.find_elements_by_tag_name('li')

In [17]:
def find_index(list, target):
    for i in range(len(list)):
        if(list[i].id == target.id):
            return i
    return -1

In [18]:
find_index(all_lis, first_res)

88

In [19]:
len(all_lis)

160

In [20]:
all_lis[88].get_attribute("id")

'resitem-0'

In [21]:
all_lis[142].get_attribute("id")

'resitem-54'

In [22]:
all_lis[143].get_attribute("id")

'yui_3_5_1_1_1509356978587_752'

In [31]:
first_res.click()

In [33]:
link = driver.find_element_by_link_text('View Image')

In [32]:
driver.window_handles

['CDwindow-e0a609a0-3c4e-48fa-9d75-3e428a57205f']

In [27]:
from selenium.webdriver.common.action_chains import ActionChains

In [37]:
ActionChains(driver).key_down(Keys.CONTROL).click(link).key_up(Keys.CONTROL).perform()

In [38]:
driver.window_handles

['CDwindow-e0a609a0-3c4e-48fa-9d75-3e428a57205f',
 'CDwindow-0e5a14ab-ae1b-4ce7-8b61-5771e4070e18']

In [42]:
driver.current_window_handle

'CDwindow-e0a609a0-3c4e-48fa-9d75-3e428a57205f'

In [44]:
driver.switch_to_window(driver.window_handles[1])

In [45]:
driver.current_url

'https://upload.wikimedia.org/wikipedia/commons/9/9d/Smitswinkel_Bay.jpg'

In [46]:
urllist = []

In [47]:
urllist.append(driver.current_url)

In [48]:
driver.close()

In [49]:
driver.window_handles

['CDwindow-e0a609a0-3c4e-48fa-9d75-3e428a57205f']

In [51]:
driver.switch_to_window(driver.window_handles[0])

In [52]:
driver.current_url

'http://images.search.yahoo.com/search/images;_ylt=AwrTcXiu9fZZ1JcAkweLuLkF;_ylc=X1MDOTYwNTc0ODMEX3IDMgRiY2sDZTZycGdwNWN2ZHRkZSUyNmIlM0QzJTI2cyUzRHEyBGZyAwRncHJpZANSMWd6aFl0eFE5dUtKbk1KM2V6dVJBBG10ZXN0aWQDbnVsbARuX3N1Z2cDMTAEb3JpZ2luA2ltYWdlcy5zZWFyY2gueWFob28uY29tBHBvcwMwBHBxc3RyAwRwcXN0cmwDBHFzdHJsAzMEcXVlcnkDYmF5BHRfc3RtcAMxNTA5MzU2OTc4BHZ0ZXN0aWQDbnVsbA--?gprid=R1gzhYtxQ9uKJnMJ3ezuRA&pvid=A1d10TEwLjJUhulTd3DH.AAyMTA0LgAAAACOSmPr&fr2=sb-top-images.search.yahoo.com&p=bay&ei=UTF-8&iscqry=&fr=sfp#id=0&iurl=http%3A%2F%2Fupload.wikimedia.org%2Fwikipedia%2Fcommons%2F9%2F9d%2FSmitswinkel_Bay.jpg&action=click'

In [ ]:
driver.close()

In [ ]:
"""
chrome_options.add_argument('--disable-gpu')
chrome_options.add_argument("start-maximized")
chrome_options.add_argument("disable-infobars")
chrome_options.add_argument("--disable-extensions")
"""